In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df= pd.read_csv("../input/sf-crime/train.csv.zip")
test_df = pd.read_csv("../input/sf-crime/test.csv.zip")

In [ ]:
leCrime =LabelEncoder()
crime = leCrime.fit_transform(train_df.Category)

In [ ]:
days = pd.get_dummies(train_df.DayOfWeek)
district = pd.get_dummies(train_df.PdDistrict)

In [ ]:
train_df.Dates=pd.to_datetime(train_df.Dates)

In [ ]:
hour = train_df.Dates.dt.hour
hour = pd.get_dummies(hour)

In [ ]:
trainDate = pd.concat([hour,days,district],axis=1)
trainDate['crime']=crime

In [ ]:
days = pd.get_dummies(test_df.DayOfWeek)
district = pd.get_dummies(test_df.PdDistrict)
test_df.Dates=pd.to_datetime(test_df.Dates)
hour = test_df.Dates.dt.hour
hour = pd.get_dummies(hour)

In [ ]:
testDate = pd.concat([hour,days,district],axis=1)

In [ ]:
# 添加犯罪的小时时间点作为特征
features = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION',
'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']
hourFea = [x for x in range(0,24)]
features = features + hourFea

In [ ]:
training,validation = train_test_split(trainDate,train_size=.8)

In [ ]:
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.linear_model import LogisticRegression
import time
from sklearn.metrics import log_loss

In [ ]:
model = BernoulliNB()
nbstart= time.time()
model.fit(training[features],training['crime'])
nbCostTime = time.time()-nbstart
predicted = np.array(model.predict_proba(validation[features]))
print("耗时: ",nbCostTime)
# print("log_loss:  ",log_loss(validation['crime'],predicted))
# print(model.score(validation[features],validation['crime']))

In [ ]:
predict = model.predict_proba(testDate)

In [ ]:
submission=pd.DataFrame(np.c_[test_df.Id,predict],
                              columns=['Id']+list(leCrime.classes_))

In [ ]:
submission['Id']=submission['Id'].astype(int)

In [ ]:
submission.to_csv("submission.csv",index=False)